In [18]:
from pathlib import Path
BASE_DIR = Path("/home/ubuntu/bird-sql-forked/save_data/qwen_mix")

GENERATION_DIR = BASE_DIR / "generations"
RESULTS_DIR = BASE_DIR / "with_results"
REWARD_DIR = BASE_DIR / "with_rewards"

### Generations

In [19]:
# load the generations jsonl file - data_gen.jsonl
import json

final_generations_data = {}
with open(GENERATION_DIR / "data_gen.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)

        if data['question_id'] not in final_generations_data:
            final_generations_data[data['question_id']] = {
                'question_id' : data['question_id'],
                'db_id' : data['db_id'],
                'question' : data['question'],
                'evidence' : data['evidence'],
                'SQL' : data['SQL'],
                'difficulty' : data['difficulty'],
                'responses' : []
            }
        curr_responses = []
        for item in data['responses']:
            curr_responses.append(data['responses'][item]['content'])
        final_generations_data[data['question_id']]['responses'] += curr_responses




In [20]:
mean_responses = 0
for key in final_generations_data:
    mean_responses += len(final_generations_data[key]['responses'])
mean_responses /= len(final_generations_data)
print(mean_responses)


249.936


In [21]:
import pandas as pd
pd.Series(final_generations_data[1473]['responses']).value_counts()

SELECT AVG(T1.Consumption) / 12 FROM yearmonth AS T1 INNER JOIN customers AS T2 ON T1.CustomerID = T2.CustomerID WHERE T2.Segment = 'SME' AND T1.Date BETWEEN 201301 AND 201312                                            15
SELECT AVG(T1.Consumption) / 12 FROM yearmonth AS T1 INNER JOIN customers AS T2 ON T1.CustomerID = T2.CustomerID WHERE T1.Date BETWEEN 201301 AND 201312 AND T2.Segment = 'SME'                                            14
SELECT AVG(T2.Consumption) / 12 FROM customers AS T1 INNER JOIN yearmonth AS T2 ON T1.CustomerID = T2.CustomerID WHERE T1.Segment = 'SME' AND T2.Date BETWEEN 201301 AND 201312                                            13
SELECT AVG(Consumption) / 12.0 AS average_monthly_consumption\nFROM yearmonth AS Y\nJOIN customers AS C ON Y.CustomerID = C.CustomerID\nWHERE C.Segment = 'SME'\nAND Y.Date BETWEEN '201301' AND '201312'                  10
SELECT AVG(T1.Consumption) / 12 FROM yearmonth AS T1 INNER JOIN customers AS T2 ON T1.CustomerID = T2.CustomerID

### With Results

In [22]:
final_results_data = {}
with open(RESULTS_DIR / "data_with_results.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)

        if data['question_id'] not in final_results_data:
            final_results_data[data['question_id']] = {
                'question_id' : data['question_id'],
                'db_id' : data['db_id'],
                'question' : data['question'],
                'evidence' : data['evidence'],
                'SQL' : data['SQL'],
                'difficulty' : data['difficulty'],
                'responses' : []
            }
        curr_responses = []
        for item in data['responses']:
            curr_responses.append((data['responses'][item]['content'], data['responses'][item]['evaluation']))
        final_results_data[data['question_id']]['responses'] += curr_responses

In [23]:
mean_responses = 0
for key in final_results_data:
    mean_responses += len(final_results_data[key]['responses'])
mean_responses /= len(final_results_data)
print(mean_responses)


134.026


In [24]:
import pandas as pd
mean_pass = 0
for key in final_results_data:
    try:
        mean_pass += pd.Series([x[1].split(':')[0] for x in final_results_data[key]['responses']]).value_counts()['pass']
    except:
        print("key : ", key)
mean_pass /= len(final_results_data)
print(mean_pass)

key :  701
key :  704
key :  518
111.804


### with rewards

In [25]:
final_rewards_data = {}
with open(REWARD_DIR / "data_reward_all.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)

        if data['question_id'] not in final_rewards_data:
            final_rewards_data[data['question_id']] = {
                'question_id' : data['question_id'],
                'db_id' : data['db_id'],
                'question' : data['question'],
                'evidence' : data['evidence'],
                'SQL' : data['SQL'],
                'difficulty' : data['difficulty'],
                'responses' : []
            }
        curr_responses = []
        for item in data['responses']:
            if data['responses'][item]['evaluation'].split(':')[0] == 'pass' and data['responses'][item]['reward'] != None:
                curr_responses.append((data['responses'][item]['content'], data['responses'][item]['evaluation'], data['responses'][item]['reward']))
        final_rewards_data[data['question_id']]['responses'] += curr_responses

In [26]:
mean_responses = 0
for key in final_rewards_data:
    mean_responses += len(final_rewards_data[key]['responses'])
mean_responses /= len(final_rewards_data)
print(mean_responses)


101.99


In [27]:
# sort the responses by reward

for key in final_rewards_data:
    final_rewards_data[key]['responses'].sort(key=lambda x: x[2], reverse=True)


In [36]:

for k in range(1, 26):
    accuracy = 0
    for key in final_rewards_data:
        all_candidates_rewards_score = [x[2] for x in final_rewards_data[key]['responses']]
        all_candidates_rewards = [x[1] for x in final_rewards_data[key]['responses']]
        all_candidates_sqls = [x[0] for x in final_rewards_data[key]['responses']]
        top_k_candidates_rewards_score = all_candidates_rewards_score[:k]
        top_k_candidates_rewards = all_candidates_rewards[:k]
        top_k_candidates_sqls = all_candidates_sqls[:k]

        if len(final_rewards_data[key]['responses']) > 0:
            if 'pass: correct' in top_k_candidates_rewards:
                accuracy += 1
    accuracy /= len(final_rewards_data)
    print("accuracy for k = ", k, " is ", accuracy)


accuracy for k =  1  is  0.698
accuracy for k =  2  is  0.714
accuracy for k =  3  is  0.72
accuracy for k =  4  is  0.728
accuracy for k =  5  is  0.734
accuracy for k =  6  is  0.74
accuracy for k =  7  is  0.744
accuracy for k =  8  is  0.748
accuracy for k =  9  is  0.748
accuracy for k =  10  is  0.75
accuracy for k =  11  is  0.752
accuracy for k =  12  is  0.758
accuracy for k =  13  is  0.762
accuracy for k =  14  is  0.764
accuracy for k =  15  is  0.764
accuracy for k =  16  is  0.766
accuracy for k =  17  is  0.766
accuracy for k =  18  is  0.774
accuracy for k =  19  is  0.778
accuracy for k =  20  is  0.782
accuracy for k =  21  is  0.784
accuracy for k =  22  is  0.786
accuracy for k =  23  is  0.792
accuracy for k =  24  is  0.792
accuracy for k =  25  is  0.794


### SQL Parsing

In [70]:
import sqlglot
from sqlglot import expressions as exp
import re

def extract_schema_linking(sql_query, dialect="sqlite"):
    try:
        # Clean up any ANSI escape codes
        clean_query = re.sub(r'\x1b\[[0-9;]*m', '', sql_query)
        parsed = sqlglot.parse_one(clean_query, dialect=dialect)

        schema_linking = {}
        tables = list(parsed.find_all(exp.Table))

        # --- Logic for single-table queries ---
        if len(tables) == 1:
            # Get the actual table name, ignoring any alias it might have
            table_name = tables[0].name 
            schema_linking[table_name] = set()

            # Find all column references
            for column in parsed.find_all(exp.Column):
                # Add every column found, as it must belong to the single table
                schema_linking[table_name].add(column.name)
        else:
            # --- Original logic for multi-table queries (JOINs) ---
            # This part requires columns to be explicitly qualified (e.g., table.column)
            alias_to_table = {t.alias_or_name: t.name for t in tables}
            
            for column in parsed.find_all(exp.Column):
                if column.table:  # Only consider columns with explicit table references
                    table_alias = column.table
                    table_name = alias_to_table.get(table_alias)
                    if table_name:
                        if table_name not in schema_linking:
                            schema_linking[table_name] = set()
                        schema_linking[table_name].add(column.name)

        # Convert sets to lists for the final output
        return {table: list(columns) for table, columns in schema_linking.items()}

    except Exception as e:
        return {"error": str(e)}

In [71]:
final_rewards_data[1471]['responses'][0][0]

"SELECT CAST(SUM(IIF(Currency = 'EUR', 1, 0)) AS REAL) / CAST(SUM(IIF(Currency = 'CZK', 1, 0)) AS REAL) AS ratio FROM customers"

In [72]:
extract_schema_linking(final_rewards_data[1471]['responses'][0][0])

{'customers': ['Currency']}

In [73]:
K=26
golden_sql_parsed = {}
candidate_sql_parsed = {}
for key in final_rewards_data:
    golden_sql_parsed[key] = extract_schema_linking(final_rewards_data[key]['SQL'])

    # aggregate the sql parsed for all the responses and union them
    merged_candidate_sql_parsed = {}
    for response in final_rewards_data[key]['responses'][:K]:
        curr_sql_parsed = extract_schema_linking(response[0])
        if 'error' not in curr_sql_parsed:
            for table, columns in curr_sql_parsed.items():
                if table not in merged_candidate_sql_parsed:
                    merged_candidate_sql_parsed[table] = set()
                merged_candidate_sql_parsed[table].update(columns)

    # Convert sets back to lists
    candidate_sql_parsed[key] = {table: list(columns) for table, columns in merged_candidate_sql_parsed.items()}

# tables matches with golden sql
tables_matches = {}
tables_accuracy = 0
tables_and_fields_accuracy = 0
for key in candidate_sql_parsed:
    golden_tables = sorted(golden_sql_parsed[key].keys())
    candidate_tables = sorted(candidate_sql_parsed[key].keys())
    table_flag = True
    for table in golden_tables:
        if table not in candidate_tables:
            table_flag = False

    if table_flag:
        tables_accuracy += 1

    table_field_flag = True
    for table in golden_tables:
        if table in candidate_tables:
            golden_fields = sorted(golden_sql_parsed[key][table])
            candidate_fields = sorted(candidate_sql_parsed[key][table])
            for field in golden_fields:
                if field not in candidate_fields:
                    table_field_flag = False
                    break
        else:
            table_field_flag = False
            break
    if table_field_flag:
        tables_and_fields_accuracy += 1

In [74]:
tables_accuracy *100.0 /500

92.6

In [75]:
tables_and_fields_accuracy * 100.0 / 500

81.6

In [76]:
K=26
golden_sql_parsed = {}
candidate_sql_parsed = {}
for key in final_rewards_data:
    golden_sql_parsed[key] = extract_schema_linking(final_rewards_data[key]['SQL'])

    # aggregate the sql parsed for all the responses and union them
    merged_candidate_sql_parsed = {}
    for response in final_rewards_data[key]['responses'][:K]:
        curr_sql_parsed = extract_schema_linking(response[0])
        if 'error' not in curr_sql_parsed:
            for table, columns in curr_sql_parsed.items():
                if table not in merged_candidate_sql_parsed:
                    merged_candidate_sql_parsed[table] = set()
                merged_candidate_sql_parsed[table].update(columns)

    # Convert sets back to lists
    candidate_sql_parsed[key] = {table: list(columns) for table, columns in merged_candidate_sql_parsed.items()}

# tables matches with golden sql
tables_matches = {}
tables_precision = 0
tables_and_fields_precision = 0
for key in candidate_sql_parsed:
    golden_tables = sorted(golden_sql_parsed[key].keys())
    candidate_tables = sorted(candidate_sql_parsed[key].keys())
    table_flag_count = 0
    for table in candidate_tables:
        if table in golden_tables:
            table_flag_count += 1
    tables_precision += table_flag_count/len(candidate_tables) if len(candidate_tables) > 0 else 0

    denom_ = 0
    table_field_flag_count = 0
    for table in candidate_tables:
        if table in golden_tables:
            golden_fields = sorted(golden_sql_parsed[key][table])
            candidate_fields = sorted(candidate_sql_parsed[key][table])
            for field in candidate_fields:
                denom_ += 1
                if field in golden_fields:
                    table_field_flag_count += 1
    tables_and_fields_precision += table_field_flag_count/denom_ if denom_ > 0 else 0


In [77]:
tables_precision *100.0 /500

85.40636363636365

In [78]:
tables_and_fields_precision * 100.0 / 500

75.94653895124479

In [90]:
golden_sql_parsed = {}
candidate_sql_parsed = {}
for key in final_rewards_data:
    golden_sql_parsed[key] = extract_schema_linking(final_rewards_data[key]['SQL'])

    # aggregate the sql parsed for all the responses and union them
    merged_candidate_sql_parsed = {}
    for response in final_rewards_data[key]['responses'][:K]:
        curr_sql_parsed = extract_schema_linking(response[0])
        if 'error' not in curr_sql_parsed:
            for table, columns in curr_sql_parsed.items():
                if table not in merged_candidate_sql_parsed:
                    merged_candidate_sql_parsed[table] = set()
                merged_candidate_sql_parsed[table].update(columns)

    # Convert sets back to lists
    candidate_sql_parsed[key] = {table: list(columns) for table, columns in merged_candidate_sql_parsed.items()}

In [92]:
# save this as a json file
import json
with open('candidate_sql_parsed.json', 'w') as f:
    json.dump(candidate_sql_parsed, f)
candidate_sql_parsed

{1471: {'customers': ['Currency']},
 1472: {'yearmonth': ['Date', 'Consumption', 'CustomerID'],
  'customers': ['CustomerID', 'Segment']},
 1473: {'yearmonth': ['consumption',
   'Consumption',
   'date',
   'Date',
   'CustomerID'],
  'customers': ['CustomerID', 'Segment']},
 1476: {'yearmonth': ['Date', 'Consumption', 'CustomerID'],
  'customers': ['Currency', 'currency', 'CustomerID']},
 1479: {'yearmonth': ['consumption',
   'customerid',
   'Consumption',
   'DATE',
   'date',
   'Date',
   'CustomerID'],
  'customers': ['Currency', 'currency', 'customerid', 'CustomerID']},
 1480: {'yearmonth': ['Consumption', 'DATE', 'date', 'Date', 'CustomerID'],
  'customers': ['CustomerID', 'Segment']},
 1481: {'customers': ['Currency',
   'currency',
   'customerid',
   'Segment',
   'segment',
   'customerID',
   'CustomerID'],
  'yearmonth': ['consumption',
   'customerid',
   'Consumption',
   'DATE',
   'customerID',
   'Date',
   'CustomerID'],
  'customer_consumption': ['CustomerID', 'S